# Notebook to pack zheng68k data from x10genomics into an AnnData (h5ad) file.  

This example follows [Zheng]() for identification of white blood cell types from single cell RNA expression data.


## Outline of process
The finetune process requires the input data to be in AnnData (h5ad) format with the cell types
as labels. If the data is not packed as AnnData, as is the case when downloading from the
10xGenomics site as explained below, it need to first be packed into one and saved to the disk.

Once the data is in AnnData format, it needs to be canonized via data and variable name transformations.
After the transformations the filtered and canonized data may be saved again in AnnData format or used directly.  This data can be used by the training process.

In the training process, each sample is transformed into a GeneFormer like ordered list of genes, which is done by sorting all the genes by binned expression level and within each bin alphabetically by the gene name (as it is used in the tokenizer).
This is done inside the task.process_model_output method, together with input preparations and tokenizations.




## preprocessing data transformations

There is a commented example of this process at the end of this notebook.  

* Filter out cell with less then 200 active genes

        scanpy.pp.filter_cells(anndata_object,min_genes=200)



* Normelize the sum of counts for each cell to a constant (1000)

        scanpy.pp.normalize_total(anndata_object,1000.)


* Move to log space (note,the data prior to this step will be in the range 0-10 so scanpy may issue a warning that the data has allready beed log-scaled)

        scanpy.pp.log1p(anndata_object,base=2)


* split the full range of values into bins and digitize the values

        bins=np.linspace(anndata_object.X.data.min(), anndata_object.X.max(),num=10)
        anndata_object.X.data=np.digitize(anndata_object.X.data, bins)

    Note that this was done over all the data, but that is not likely to cause any bleeding from the test sets



This notebook assumes that it is run from the `bmfm-mammal-release/mammal/examples/scrna_cell_type` directory ("the current directory").  

In [ ]:
# check the current directory. 
# Notice that the `biomed-multi-alignment` will probably be placed in a different location on your system.

# this is just a fancy pwd, replace with !pwd if you need
!print -D $PWD

~/git/biomed-multi-alignment/mammal/examples/scrna_cell_type/data



## Obtaining the source data:
The main data is available online, for example in the [10xGenomics](https://www.10xgenomics.com/) cite.  The labels are based on the data in [LINK](https://www.10xgenomics.com/datasets/fresh-68-k-pbm-cs-donor-a-1-standard-1-1-0)

From this download the file `fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz` and place it in this directory. 

In [ ]:
!ls -sh

total 243096
    32 Zheng68k_data_prep.ipynb
     0 filtered_matrices_mex
243056 fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz
     8 process_h5ad_data.py


#### Unzip the file.

In [ ]:
!tar -xzvf fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz 

x filtered_matrices_mex/
x filtered_matrices_mex/hg19/
x filtered_matrices_mex/hg19/barcodes.tsv
x filtered_matrices_mex/hg19/genes.tsv
x filtered_matrices_mex/hg19/matrix.mtx


#### Download the labels file from a git repository in https://github.com/scverse/scanpy_usage

In [ ]:
!wget https://raw.githubusercontent.com/scverse/scanpy_usage/refs/heads/master/170503_zheng17/data/zheng17_bulk_lables.txt

--2025-02-19 12:50:16--  https://raw.githubusercontent.com/scverse/scanpy_usage/refs/heads/master/170503_zheng17/data/zheng17_bulk_lables.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1348927 (1.3M) [text/plain]
Saving to: ‘zheng17_bulk_lables.txt’

zheng17_bulk_lables 100%[===================>]   1.29M  3.78MB/s    in 0.3s    

2025-02-19 12:50:16 (3.78 MB/s) - ‘zheng17_bulk_lables.txt’ saved [1348927/1348927]



  You should now have a directory called `filtered_matrices_mex/hg19` 

In [ ]:
!ls -shR --color=never


total 245736
    32 Zheng68k_data_prep.ipynb
     0 filtered_matrices_mex
243056 fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz
     8 process_h5ad_data.py
  2640 zheng17_bulk_lables.txt

./filtered_matrices_mex:
total 0
0 hg19

./filtered_matrices_mex/hg19:
total 993144
  2280 barcodes.tsv	  1600 genes.tsv	989264 matrix.mtx


The output should be something like

```total 245736
    32 Zheng68k_data_prep.ipynb
     0 filtered_matrices_mex
243056 fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz
     8 process_h5ad_data.py
  2640 zheng17_bulk_lables.txt

./filtered_matrices_mex:
total 0
0 hg19

./filtered_matrices_mex/hg19:
total 993144
  2280 barcodes.tsv	  1600 genes.tsv	989264 matrix.mtx
  ```

## Pack the data into an AnnData file 

In [ ]:
import anndata 
# from collections import Counter
import pandas as pd
# import matplotlib.pyplot as plt
# import math
import numpy as np
# from scipy.sparse import csr_matrix
from scipy.io import mmread
import scanpy as sc

#### Read the scRNA matrix from a file


In [ ]:
mmx = mmread("filtered_matrices_mex/hg19/matrix.mtx")


#### Create an AnnData object wrapping the read data

Notice that this code transposes the data to the correct direction

In [ ]:
 
anndata_object = anndata.AnnData(X=mmx.transpose().tocsr())
print(anndata_object.X.shape)

(68579, 32738)


In [ ]:
# Cell identifiers
barcodes = pd.read_csv("filtered_matrices_mex/hg19/barcodes.tsv",header=None,sep="\t")
# names of genes
genes = pd.read_csv("filtered_matrices_mex/hg19/genes.tsv",header=None,sep="\t")


In [ ]:
# cell types (this is actualy just one column)
cell_type_lables = pd.read_csv("zheng17_bulk_lables.txt",header=None,sep="\t")

In [ ]:

# use the gene names as variable names in the AnnData object
anndata_object.var_names=genes[1]

# use the cell barcodes as names for the samples
anndata_object.obs_names=barcodes[0]



In [ ]:

# use cell types as labels for the samples
anndata_object.obs['celltype']=cell_type_lables.squeeze().to_numpy()

In [ ]:
# Save result anndata object to disk
anndata_object.write_h5ad("Zheng_68k.h5ad")

This is a standard AnnData file, and can be replaced by your own data.

The code requires a simple preprocessing stage to get it to a standard form.  [process_h5ad_data.py] is a script that can be used to run the preprocessing stage.

This script can be used to filter and process the AnnData

    > python process+h5ad_data.py --input-h5ad-file Zheng_68k.h5ad --output-h5ad-file Zheng_68k_filtered.h5ad

The parameters of the processing can be controlled from the command line.  Use

    >  python process+h5ad_data.py --help

for a list of available parameters

### And the annData file is ready in the data directory.

---------


This code, which can also be found in the  [process_h5ad_data.py] script, explains and lists the steps needed for processing the input.

### process the data - filter out cells with shallow reads, normalize depth and change to log scale of about 0-10 (log_2(1001)~=10)

    sc.pp.filter_cells(anndata_object,min_genes=200)
    sc.pp.normalize_total(anndata_object,1000.)
    sc.pp.log1p(anndata_object,base=2)


    anndata_object.X.data.min(), anndata_object.X.max()
### split range to bins - more or less 0-1,1-2,2-3,..10
    bins=np.linspace(anndata_object.X.data.min(), anndata_object.X.max(),num=11)
    print(bins)


### convert the counts to bins
    anndata_object.X.data=np.digitize(anndata_object.X.data, bins)

### Save result anndata object to disk
    anndata_object.write_h5ad("Zheng68k_filtered.h5ad")




### Inside the data pipeline the sample is converted to a GeneFormer inspired ordered list of genes.
### Note that this is not part of the preprocessing and is only here to explain the process.

    def convert_to_double_sorted_geneformer_sequence(anndata_object):

        # the genes are sorted by expression bin (descending) and within the bin by the gene names.    
        return [a[1] for a in sorted(zip(-anndata_object.X.data,anndata_object.var_names.to_numpy()[anndata_object.X.indices]))]

